##텍스트 전처리(Text Preprocessing)

컴퓨터가 자연어를 이해하도록 학습시키기 위해서 시소러스부터 word2vec까지 다양한 형태의 기법이 사용된다. 위 과정에서 효율적인 연산을 돕기 위해 corpus(말뭉치: 대량의 텍스트 데이터)를 전처리 하는 과정이 필요하다. 각 단계는 corpus를 자르고, 정제하고, 통합함으로써 의미를 파악하는데 핵심적인 요소만을 남기는데 목적을 두고 있다.

NPL 학습의 가장 큰 착안점은 단어의 의미가 맥락에서(주변 단어의 관계에서) 형성된다는 "분포 가설"이다. 따라서 문장을 분리하고, 각 단어의 위치를 인덱싱하는 과정이 필요하다.

###토큰화(Tokenization)

토큰화는 주어진 텍스트 데이터를 토큰이라 불리는 작은 단위로 쪼개는 작업을 이른다. 토큰의 크기는 상황에 따라 다르지만, 보통 의미를 가지는 언어의 단위(단어, 어구, 문장 등)를 사용한다.

####단어 토큰화

의미를 가지는 최소 단위인 단어를 기준으로 나누는 것이다. 언어의 특성마다 다르지만, 영어의 경우 뛰어쓰기에 사용되는 " "(공백)을 이용해서 나눌 수 있기 때문에 간단히 구현이 가능하다.

In [2]:
text="I like to learn Python."
text= text.replace('.', ' .')
t_list=text.split(" ")
t_list

['I', 'like', 'to', 'learn', 'Python', '.']

In [10]:
text="I like to learn Python."
text=text.lower()
import re
re.split('(\W+)',text)

['i', ' ', 'like', ' ', 'to', ' ', 'learn', ' ', 'python', '.', '']

한국어의 경우 조사를 붙여쓰는 교착어로 어절(띄어쓰기의 단위)이 언제나 단어와 일치하지 않기 때문에 이에 유의하여 토큰화를 진행해야 한다. 

어절이 아닌 형태소 단위로 나눔으로써 이를 해결할 수 있다. KoNLPy라는 파이썬 패키지를 이용해 간단히 형태소 단위의 토큰화가 가능하다.

In [11]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 123 kB/s 
     |████████████████████████████████| 86 kB 6.4 MB/s 
     |████████████████████████████████| 448 kB 44.4 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [14]:
from konlpy.tag import Okt
okt=Okt()
print(okt.morphs("한국어로 하는 자연어 처리는 즐거워!"))

['한국어', '로', '하는', '자연어', '처리', '는', '즐거워', '!']


####문장 토큰화

문장 토큰화는 corpus가 하나의 글, 즉 여러 문장인 경우 이를 나누기 위한 작업이다. 문장은 맥락을 가지는 최소의 단위로 한 문장에서의 단어의 절대적, 상대적 위치를 파악하는 것은 그 단어의 성질을 이해하는데 큰 도움이 된다. 가령 주어로 쓰일 수 있는 'I', 'You', 'She'등은 한 문장의 맨 앞에 놓일 가능성이 크다. 기본적으로 문장의 종결을 알리는 기호들('.', '?', '!')을 이용한 분류가 가능하다. 하지만, 위 기호들이 언제나 문장의 끝을 알리는 것은 아니므로 주의해야 한다.

마침표는 약어의 표시와 문장종결의 표시 두 역할을 수행한다. 따라서 두 경우를 구별하기 위한 이진 분류기가 필요하다. 마침표 주변에 숫자, @, 또 다른 마침표의 유무 등을 확임함으로써 이 둘을 구별할 수 있다. 

##정제(Cleaning) & 정규화(Normalization)

####정제(Cleaning)

정제는 주어진 corpus에서 문장의 의미 파악에 방해가 되는 혹은 필요가 없는 노이즈 데이터를 제거하는 작업이다. 언어의 특성에 따라 다르겠지만 그 기준으로 낮은 등장 빈도, 짤은 길이 등이 있다. 예를 들어 영어의 관사('a', 'an')등은 특별한 의미를 가지지 않으므로 연산의 효율성을 위해서 제거하는 것이 좋다. 

####정규화(Normalization)

정규화는 표현 방법이 다른 단어들을 통합시키는 과정이다. 줄임말이나 영어의 대소문자를 통일 시키는 것이 대표적이다. 예로, 서강대=>서강대학교 로 변환하는 과정 등이 있다. 이를 위해서는 단어의 핵심의미를 가지는 부분들을 추출하는 작업들이 필요하다.

####어간 추출(Stemming) & 표제어 추출(Lemmatization)

어간 추출과 표제어 추출은 corpus에 존재하는 단어의 가짓수를 줄이기 위한 작업이다. 다른 형태이지만, 동일한 의미를 가리키게 하는 핵심적인 부분들을 추출함으로써 단어를 통합시킬 수 있다. 

표제어 추출은 다양한 형태로 활용된 단어들을 기본형의 단어로 매칭시키는 작업니다. 가령 명사의 복수형 어미 '-es', '-s'를 제거하거나 동사의 과거형을 기본형으로 바꾸어주는 과정이 있다. 이때 각 단어의 품사를 아는 것이 큰 도움이 된다는 점에서, 품사 태깅이 중요하다. 

어간 추출은 하나의 단어를 의미구로 자르는 작업을 한다. 이는 한국어의 형태소 분석과 비슷한 역할을 하는데, 새로운 단어의 의미를 기존에 알고 있는 단어로 분리해 연관지을 수 있도록 한다. 

###불용어(Stopword)

불용어는 의미 파악에 크게 도움되지 않는 단어 토큰을 제거하는 작업이다. 자주 등장하지만 학습에 크게 도움되지 않는 단어들을 지정해 삭제할 수 있다. 

###원-핫 인코딩(One-Hot Encoding)

원-핫 인코딩은 컴퓨터가 텍스트들의 관계를 연산하는데 용이하기 위해 토큰에 ID를 부여하는 전처리를 말한다. 단어 토큰화의 경우 문장에서의 위치로 라벨링하여 나타낸다. 

In [5]:
text="I like to learn Python."
text=text.lower()
text= text.replace('.', ' .')
t_list=text.split(" ")

word_to_id={}
id_to_word={}

for word in t_list:
    if word not in word_to_id:
        new_id=len(word_to_id)
        word_to_id[word]=new_id
        id_to_word[new_id]=word

print(word_to_id)
print(id_to_word)

{'i': 0, 'like': 1, 'to': 2, 'learn': 3, 'python': 4, '.': 5}
{0: 'i', 1: 'like', 2: 'to', 3: 'learn', 4: 'python', 5: '.'}


이러한 전처리를 통해서 최빈 단어 순으로 나열하거나, 동시발생행렬을 만들 수 있다. 동시발생 행렬이란 하나의 단어 주변에 다른 단어가 몇번 나타나는지를 표시하는 행렬이다. 주변의 정의(window size), 좌우 방향, 문장의 시작과 끝 등의 요소를 고려해 주변 단어를 집계한다.